In [1]:
import sys
import os
import glob

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patheffects as patheffects
import pandas as pd

from scipy.spatial.distance import cdist, pdist, squareform
from scipy.special import factorial, euler
from scipy.interpolate import interp1d
import scipy.stats 
from scipy.optimize import curve_fit
# import dateutil
# import ftplib
# import io
# import datetime
# import time
# import requests
import statsmodels.formula.api as smformula
import statsmodels.api as sm

import itertools
from collections import OrderedDict

from IPython.display import display, HTML, Markdown
%matplotlib notebook
plt.style.use('seaborn-paper')

C:\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [33]:
fn = r'C:\Users\Travis Collier\Desktop\proj\ddtool\testfiles\USDA-WS6\20148981___Over_the_last_week_2018_04_30_22_07_39_UTC_1_pt1.csv'

df = pd.read_csv(fn, parse_dates=['Date']).dropna()
df

,Line#,Date,"Temperature (S-THB 20148981:20147576-1), *C, USDA-WS6","PAR (S-LIA 20148981:20140001-1), uE, USDA-WS6","RH (S-THB 20148981:20147576-2), %, USDA-WS6","Temperature (S-TMB 20148981:20130466-1), *C, USDA-WS6","Solar Radiation (S-LIB 20148981:20142891-1), W/m^2, USDA-WS6","Dew Point (S-THB 20148981:20147576-3), *C, USDA-WS6","Water Content (S-SMD 20148981:20152203-1), m^3/m^3, USDA-WS6","Rain (S-RGB 20148981:20148771-1), mm, USDA-WS6","Battery (RX3000 BATTERY 20148981:20148981-B), V, USDA-WS6"
0,1.0,2018-04-23 22:10:00,23.64,2226.0,85.9,22.32,1094.0,21.17,0.4298,0.0,4.433
1,2.0,2018-04-23 22:15:00,24.27,1146.0,83.8,22.27,559.0,21.38,0.4298,0.0,4.427
2,3.0,2018-04-23 22:20:00,24.10,1331.0,82.0,22.32,648.0,20.86,0.4298,0.0,4.420
3,4.0,2018-04-23 22:25:00,24.36,2209.0,81.2,22.35,1078.0,20.96,0.4298,0.0,4.412
4,5.0,2018-04-23 22:30:00,24.87,1599.0,79.2,22.39,768.0,21.05,0.4291,0.0,4.405
5,6.0,2018-04-23 22:35:00,25.36,2029.0,78.4,22.44,989.0,21.35,0.4298,0.0,4.436
6,7.0,2018-04-23 22:40:00,24.68,1301.0,75.0,22.44,626.0,19.97,0.4291,0.0,4.486
7,8.0,2018-04-23 22:45:00,23.57,1251.0,81.1,22.37,601.0,20.16,0.4298,0.0,4.475
8,9.0,2018-04-23 22:50:00,23.79,1134.0,77.8,22.27,549.0,19.70,0.4298,0.0,4.454
9,10.0,2018-04-23 22:55:00,24.32,1384.0,79.5,22.25,668.0,20.57,0.4298,0.0,4.442


In [55]:
tcol = [x for x in df.columns if x.startswith('Temperature ')]
if len(tmp) < 1:
    print("ERROR: Temperature column not found", file=sts.stderr)
else:
    tmp = [x.strip() for x in tcol[0].split(',')]
    station = tmp[-1]
    print(tmp, station)
    
    t = df.loc[:,['Date',tcol[0]]]
    t.set_index('Date', inplace=True)
    t.columns = ['temperature']
    t.sort_index(inplace=True)
    #t['station'] = station
    first = t.index[0]
    last = t.index[-1]
    
    print(station, first, last)
t.head()

['Temperature (S-THB 20148981:20147576-1)', '*C', 'USDA-WS6'] USDA-WS6
USDA-WS6 2018-04-23 22:10:00 2018-04-24 23:55:00


,temperature
Date,
2018-04-23 22:10:00,23.64
2018-04-23 22:15:00,24.27
2018-04-23 22:20:00,24.10
2018-04-23 22:25:00,24.36
2018-04-23 22:30:00,24.87


In [62]:
list(range(3,1,-1))

[3, 2]